## Imports

In [1]:
import pandas as pd
import numpy as np

## Read data

In [115]:
filtered_diag_df = pd.read_csv('filtered data/filtered_diagnosis_df.csv', index_col=0)
filtered_drug_adm_df = pd.read_csv('data/drugs_adm.csv', index_col=0)
filtered_drug_med_df = pd.read_csv('filtered data/filtered_drug_med_df.csv', index_col=0)
filtered_drug_recom_df = pd.read_csv('filtered data/filtered_drug_recom_df.csv', index_col=0)
filtered_jugular_df = pd.read_csv('filtered data/filtered_jugular_df.csv', index_col=0)
filtered_labs_df = pd.read_csv('filtered data/filtered_labs_df.csv', index_col=0)
filtered_limbs_df = pd.read_csv('filtered data/filtered_limbs_df.csv', index_col=0)
filtered_lungs_df = pd.read_csv('filtered data/filtered_lungs_df.csv', index_col=0)
filtered_measures_df = pd.read_csv('filtered data/new_filtered_measures.csv', index_col=0)

/var/folders/wc/nyw2632d74j_dl64800j39mw0000gn/T/ipykernel_27831/78979866.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  filtered_labs_df = pd.read_csv('filtered data/filtered_labs_df.csv', index_col=0)
/var/folders/wc/nyw2632d74j_dl64800j39mw0000gn/T/ipykernel_27831/78979866.py:9: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  filtered_measures_df = pd.read_csv('filtered data/new_filtered_measures.csv', index_col=0)


In [28]:
## Add patients information
patients_info = pd.read_csv('data/patients.csv')
patients_info['birth_date'] = pd.to_datetime(patients_info['birth_date'], format='%Y-%m-%d %H:%M:%S.%f')
patients_info['admission_time'] = pd.to_datetime(patients_info['admission_time'], format='%Y-%m-%d %H:%M:%S.%f')
patients_info['discharge_time'] = pd.to_datetime(patients_info['discharge_time'], format='%Y-%m-%d %H:%M:%S.%f')

patients_info['birth_date'] = pd.to_datetime(patients_info['birth_date'], format='mixed')
patients_info['admission_time'] = pd.to_datetime(patients_info['admission_time'], format='mixed')
patients_info['discharge_time'] = pd.to_datetime(patients_info['discharge_time'], format='mixed')
patients_info['Age'] = patients_info['admission_time'].dt.year - patients_info['birth_date'].dt.year
gender_mapping = {'female': 1, 'male': 0}
patients_info['Gender'] = patients_info['gender'].map(gender_mapping)
#patients_info['Length of stay'] = (patients_info['discharge_time'] - patients_info['admission_time']).dt.days
# patients_info.sort_values(by=['patient_id', 'admission_time'], inplace=True)
#patients_info['readmission'] = patients_info.duplicated(subset='patient_id', keep='first').astype(int)

In [9]:
# Sort DataFrame by patient_id and admission_time
patients_info.sort_values(by=['patient_id', 'admission_time'], inplace=True)
# Add a column for readmission
patients_info['Readmission (30 days)'] = 0
# Iterate over rows and check for readmission
for i in range(1, len(patients_info)):
    if patients_info['patient_id'].iloc[i] == patients_info['patient_id'].iloc[i - 1]:
        discharge_to_admission_gap = patients_info['admission_time'].iloc[i] - patients_info['discharge_time'].iloc[i - 1]
        if discharge_to_admission_gap <= pd.Timedelta(days=30):
            patients_info.at[patients_info.index[i], 'Readmission (30 days)'] = 1

In [11]:
patients_info[['case_no', 'Gender', 'Age', 'Readmission (30 days)']].to_csv('data_for_model/patients_info.csv')

## Measurements

In [116]:
measures_feats_df = filtered_measures_df[['case_no', 'MEASURE_CODE', 'RESULT_NUMERIC', 'MEASURE_NAME', 'MEASURE_TIME']]
measures_feats_df.drop_duplicates(inplace=True)
measures_feats_df['case_no'] = measures_feats_df['case_no'].astype(str)
measures_groups = pd.read_excel('measures_groups.xlsx')

/var/folders/wc/nyw2632d74j_dl64800j39mw0000gn/T/ipykernel_27831/1015471249.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  measures_feats_df.drop_duplicates(inplace=True)
/var/folders/wc/nyw2632d74j_dl64800j39mw0000gn/T/ipykernel_27831/1015471249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  measures_feats_df['case_no'] = measures_feats_df['case_no'].astype(str)


In [117]:
# remove measurements with count < 50
measures_names = measures_feats_df['MEASURE_NAME'].value_counts().reset_index().rename(columns={'MEASURE_NAME' : 'count', 'index': 'MEASURE_NAME'})
measures_names_to_remain = measures_names.loc[measures_names['count'] > 50]['MEASURE_NAME']
measures_feats_df = measures_feats_df.loc[measures_feats_df['MEASURE_NAME'].isin(measures_names_to_remain)]

In [118]:
merged_measures = pd.merge(measures_feats_df, measures_groups[['MEASURE_CODE', 'MEASURE_GROUP']], on='MEASURE_CODE', how='left')

In [ ]:
first_measurements = merged_measures.groupby(['case_no', 'MEASURE_GROUP'], as_index=False).first()
first_pivot_measures_df = first_measurements.pivot_table(index=['case_no'], values='RESULT_NUMERIC', columns='MEASURE_GROUP')
first_pivot_measures_df = first_pivot_measures_df.add_suffix('_first')

In [ ]:
last_measurements = merged_measures.groupby(['case_no', 'MEASURE_GROUP'], as_index=False).last()
last_pivot_measures_df = last_measurements.pivot_table(index=['case_no'], values='RESULT_NUMERIC', columns='MEASURE_GROUP')
last_pivot_measures_df = last_pivot_measures_df.add_suffix('_last')

In [ ]:
mean_pivot_measures_df = merged_measures.pivot_table(index=['case_no', 'MEASURE_GROUP'], values='RESULT_NUMERIC', aggfunc='mean').reset_index()
mean_pivot_measures_df = mean_pivot_measures_df.pivot_table(index=['case_no'], values='RESULT_NUMERIC', columns='MEASURE_GROUP')
mean_pivot_measures_df = mean_pivot_measures_df.add_suffix('_mean')

In [122]:
std_pivot_measures_df = merged_measures.pivot_table(index=['case_no', 'MEASURE_GROUP'], values='RESULT_NUMERIC', aggfunc='std').reset_index()
std_pivot_measures_df = std_pivot_measures_df.pivot_table(index=['case_no'], values='RESULT_NUMERIC', columns='MEASURE_GROUP')
std_pivot_measures_df = std_pivot_measures_df.add_suffix('_std')

In [ ]:
min_measurements = merged_measures.groupby(['case_no', 'MEASURE_GROUP'], as_index=False).min()
min_pivot_measures_df = min_measurements.pivot_table(index=['case_no'], values='RESULT_NUMERIC', columns='MEASURE_GROUP')
min_pivot_measures_df = min_pivot_measures_df.add_suffix('_min')

In [ ]:
max_measurements = merged_measures.groupby(['case_no', 'MEASURE_GROUP'], as_index=False).max()
max_pivot_measures_df = max_measurements.pivot_table(index=['case_no'], values='RESULT_NUMERIC', columns='MEASURE_GROUP')
max_pivot_measures_df = max_pivot_measures_df.add_suffix('_max')

In [125]:
# systolic-diastolic more features
df_filtered = merged_measures[merged_measures['MEASURE_NAME'].isin(['סיסטולי', 'דיאסטולי'])]
pivot_table = df_filtered.pivot_table(index=['case_no', 'MEASURE_TIME'], columns='MEASURE_NAME', values='RESULT_NUMERIC').reset_index()
pivot_table['sys_dias_diff'] = pivot_table['סיסטולי'] - pivot_table['דיאסטולי']
pivot_table['MAP'] = ((pivot_table['סיסטולי'] * 2) + (pivot_table['דיאסטולי'])) / 3
sys_dias_diff = pivot_table[['case_no', 'sys_dias_diff', 'MAP']]
minimum_diff = sys_dias_diff.groupby(['case_no'])['sys_dias_diff'].min().reset_index().rename(columns={'sys_dias_diff': 'sys_dias_diff_min'})
maximum_diff = sys_dias_diff.groupby(['case_no'])['sys_dias_diff'].max().reset_index().rename(columns={'sys_dias_diff': 'sys_dias_diff_max'})
mean_diff = sys_dias_diff.groupby(['case_no'])['sys_dias_diff'].mean().reset_index().rename(columns={'sys_dias_diff': 'sys_dias_diff_mean'})

In [ ]:
# merge all tables
merged_measures_df = pd.merge(first_pivot_measures_df, last_pivot_measures_df, on='case_no', how='outer')
merged_measures_df = pd.merge(merged_measures_df, mean_pivot_measures_df, on='case_no', how='outer')
merged_measures_df = pd.merge(merged_measures_df, min_pivot_measures_df, on='case_no', how='outer')
merged_measures_df = pd.merge(merged_measures_df, max_pivot_measures_df, on='case_no', how='outer')
merged_measures_df = pd.merge(merged_measures_df, std_pivot_measures_df, on='case_no', how='outer')
merged_measures_df = pd.merge(merged_measures_df, minimum_diff, on='case_no', how='outer')
merged_measures_df = pd.merge(merged_measures_df, maximum_diff, on='case_no', how='outer')
merged_measures_df = pd.merge(merged_measures_df, mean_diff, on='case_no', how='outer')

In [127]:
merged_measures_df.to_csv('data_for_model/measurements_df.csv')

## Lungs

In [137]:
lungs_feats_df = filtered_lungs_df[['case_no', 'Lungs_Good_Air_Intake', 'Lungs_Reduced_Air_Intake', 'Lungs_Moist_Crackles', 'Lungs_Wheezing', 'Lungs_Bronchial_Breathing', 'Lungs_Opacity_Knocks', 'Lungs_Elongated_Akspirium']]
# if there is a case no with 1 and 0, it takes the 1
lungs_feats_df = lungs_feats_df.groupby('case_no').agg('max').reset_index()

In [138]:
lungs_feats_df.to_csv('data_for_model/lungs_df.csv')

## Limbs

In [95]:
limbs_feats_df = filtered_limbs_df[['case_no', 'LIMBS_Without_Peripheral_Edema', 'LIMBS_Peripheral_Edema', 'LIMBS_PLACE']]
# if there is a case no with 1 and 0, it takes the 1
limbs_feats_df = limbs_feats_df.groupby('case_no').agg('max').reset_index()

In [98]:
limbs_feats_df.to_csv('data_for_model/limbs_df.csv')

## Jugular

In [99]:
jugular_feats_df = filtered_jugular_df.drop(columns=['patient_id', 'time_from_admission', 'admission_time', 'PhysicalTests_TIME',
                                                     'PhysicalTests_DEPT_id', 'JugularVeins_Not_proper_Remarks', 'JugularVeins_Details',
                                                     'JugularVeins_Not_proper_There_is_congestion_Vreede',
                                                     'JugularVeins_Not_proper_Evaluation_of_venous_pressure',
                                                     'JugularVeins_Not_proper_The_phto_jugular_reflex'])
# if there is a case no with 1 and 0, it takes the 1
jugular_feats_df = jugular_feats_df.groupby('case_no').agg('max').reset_index()

In [100]:
jugular_feats_df.to_csv('data_for_model/jugular_df.csv')

## Diagnosis

In [ ]:
diag_feats_df = filtered_diag_df[['case_no', 'DIAG_CODE', 'DESCRIPTION']]
diag_feats_df.drop_duplicates(inplace=True)
diag_groups = pd.read_excel('diag_groups.xlsx')

In [75]:
diag_counts = diag_feats_df['DESCRIPTION'].value_counts().reset_index().rename(columns={'DESCRIPTION' : 'count', 'index': 'DESCRIPTION'})
diag_names = diag_counts.loc[diag_counts['count'] > 50]['DESCRIPTION']
diag_feats_df = diag_feats_df.loc[diag_feats_df['DESCRIPTION'].isin(diag_names)]

In [ ]:
merged_diag = pd.merge(diag_feats_df, diag_groups, on='DESCRIPTION', how='left')

In [77]:
diag_dummies = pd.get_dummies(merged_diag[['case_no', 'DIAG_GROUP']], columns=['DIAG_GROUP'], dtype=int)
diag_dummies = diag_dummies.groupby('case_no').agg('sum').reset_index()

In [79]:
diag_dummies.to_csv('data_for_model/diag_df.csv')

## Labs

In [128]:
%%time
# taking specific lab tests that relevant to AKI
filtered_labs_df['case_no'] = filtered_labs_df['case_no'].astype(str)
filtered_labs_df = filtered_labs_df.loc[(filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('urine')) | (filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('Creatinine')) | (filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('BUN')) |
(filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('Albumin')) | (filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('BNP')) |
(filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('LDL')) | (filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('Chlorid'))
| (filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('RDW')) | (filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('Furosemide')) |
(filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('CBC')) |(filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('HDL')) |
(filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('Triglycerides')) | (filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('Cholesterol')) |
(filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('CK')) | (filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('Potassium')) | (filtered_labs_df['LAB_EXAMINATION_NAME'] == 'Glucose-BL') |(filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('Sodium')) | (filtered_labs_df['LAB_EXAMINATION_NAME'] == 'Calcium-BL') | (filtered_labs_df['LAB_EXAMINATION_NAME'] == 'WBC') | (filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('Troponin'))
| (filtered_labs_df['LAB_EXAMINATION_NAME'].str.contains('Glucose-UR'))]

CPU times: user 4.63 s, sys: 439 ms, total: 5.07 s
Wall time: 5.46 s


In [ ]:
first_labs = filtered_labs_df.groupby(['case_no', 'LAB_EXAMINATION_NAME'], as_index=False).first()
first_pivot_labs_df = first_labs.pivot_table(index=['case_no'], values='exam_numeric_result', columns='LAB_EXAMINATION_NAME')
first_pivot_labs_df = first_pivot_labs_df.add_suffix('_first')
# remove columns with less than 50 values
first_counts = first_pivot_labs_df.notna().sum().reset_index()
cols = first_counts.loc[first_counts[0] >= 50]['LAB_EXAMINATION_NAME']
first_pivot_labs_df = first_pivot_labs_df[cols]

In [ ]:
last_labs = filtered_labs_df.groupby(['case_no', 'LAB_EXAMINATION_NAME'], as_index=False).last()
last_pivot_labs_df = last_labs.pivot_table(index=['case_no'], values='exam_numeric_result', columns='LAB_EXAMINATION_NAME')
last_pivot_labs_df = last_pivot_labs_df.add_suffix('_last')
# remove columns with less than 50 values
last_counts = last_pivot_labs_df.notna().sum().reset_index()
cols = last_counts.loc[last_counts[0] >= 50]['LAB_EXAMINATION_NAME']
last_pivot_labs_df = last_pivot_labs_df[cols]

In [ ]:
mean_pivot_labs_df = filtered_labs_df.pivot_table(index=['case_no', 'LAB_EXAMINATION_NAME'], values='exam_numeric_result', aggfunc='mean').reset_index()
mean_pivot_labs_df = mean_pivot_labs_df.pivot_table(index=['case_no'], values='exam_numeric_result', columns='LAB_EXAMINATION_NAME')
mean_pivot_labs_df = mean_pivot_labs_df.add_suffix('_mean')
mean_counts = mean_pivot_labs_df.notna().sum().reset_index()
cols = mean_counts.loc[mean_counts[0] >= 50]['LAB_EXAMINATION_NAME']
mean_pivot_labs_df = mean_pivot_labs_df[cols]

In [133]:
std_pivot_labs_df = filtered_labs_df.pivot_table(index=['case_no', 'LAB_EXAMINATION_NAME'], values='exam_numeric_result', aggfunc='std').reset_index()
std_pivot_labs_df = std_pivot_labs_df.pivot_table(index=['case_no'], values='exam_numeric_result', columns='LAB_EXAMINATION_NAME')
std_pivot_labs_df = std_pivot_labs_df.add_suffix('_std')
std_counts = std_pivot_labs_df.notna().sum().reset_index()
cols = std_counts.loc[std_counts[0] >= 50]['LAB_EXAMINATION_NAME']
std_pivot_labs_df = std_pivot_labs_df[cols]

In [ ]:
max_pivot_labs_df = filtered_labs_df.pivot_table(index=['case_no', 'LAB_EXAMINATION_NAME'], values='exam_numeric_result', aggfunc='max').reset_index()
max_pivot_labs_df = max_pivot_labs_df.pivot_table(index=['case_no'], values='exam_numeric_result', columns='LAB_EXAMINATION_NAME')
max_pivot_labs_df = max_pivot_labs_df.add_suffix('_max')
max_counts = max_pivot_labs_df.notna().sum().reset_index()
cols = max_counts.loc[max_counts[0] >= 50]['LAB_EXAMINATION_NAME']
max_pivot_labs_df = max_pivot_labs_df[cols]

In [ ]:
min_pivot_labs_df = filtered_labs_df.pivot_table(index=['case_no', 'LAB_EXAMINATION_NAME'], values='exam_numeric_result', aggfunc='min').reset_index()
min_pivot_labs_df = min_pivot_labs_df.pivot_table(index=['case_no'], values='exam_numeric_result', columns='LAB_EXAMINATION_NAME')
min_pivot_labs_df = min_pivot_labs_df.add_suffix('_min')
min_counts = min_pivot_labs_df.notna().sum().reset_index()
cols = min_counts.loc[min_counts[0] >= 50]['LAB_EXAMINATION_NAME']
min_pivot_labs_df = min_pivot_labs_df[cols]

In [ ]:
# merge all tables
merged_labs_df = pd.merge(first_pivot_labs_df, last_pivot_labs_df, on='case_no', how='outer')
merged_labs_df = pd.merge(merged_labs_df, mean_pivot_labs_df, on='case_no', how='outer')
merged_labs_df = pd.merge(merged_labs_df, std_pivot_labs_df, on='case_no', how='outer')
merged_labs_df = pd.merge(merged_labs_df, max_pivot_labs_df, on='case_no', how='outer')
merged_labs_df = pd.merge(merged_labs_df, min_pivot_labs_df, on='case_no', how='outer')

In [137]:
merged_labs_df.to_csv('data_for_model/labs_df.csv')

# Drugs

In [ ]:
drugs_info = pd.read_excel('data/Dataset description_CHF.xlsx', sheet_name=14, header=1)
drugs_groups = drugs_info[['Medication brand name', 'Mapped ATC code', 'Anatomical main group (ATC1)']]
drugs_groups.rename(columns={'Medication brand name': 'DRUG_COMMERCIAL_NAME'}, inplace=True)

In [ ]:
filtered_drug_med_df['case_no'] = filtered_drug_med_df['case_no'].astype(str)
filtered_med_df_with_groups = pd.merge(drugs_groups, filtered_drug_med_df, on='DRUG_COMMERCIAL_NAME', how='right')

In [ ]:
filtered_adm_df_with_groups = pd.merge(drugs_groups, filtered_drug_adm_df, on='DRUG_COMMERCIAL_NAME', how='right')
filtered_adm_df_with_groups.rename(columns={'CASE_NO': 'case_no'}, inplace=True)

In [ ]:
filtered_recom_df_with_groups = pd.merge(drugs_groups, filtered_drug_recom_df, on='DRUG_COMMERCIAL_NAME', how='right')

In [246]:
filtered_med_df_with_groups = filtered_med_df_with_groups[['case_no', 'Anatomical main group (ATC1)']].rename(columns={'Anatomical main group (ATC1)': 'MED_GROUP'})
filtered_adm_df_with_groups = filtered_adm_df_with_groups[['case_no', 'Anatomical main group (ATC1)']].rename(columns={'Anatomical main group (ATC1)': 'MED_GROUP'})

In [ ]:
filtered_adm_df_with_groups.rename(columns={'Anatomical main group (ATC1)': 'MED_GROUP'}, inplace=True)

In [248]:
med_df_dummies = pd.get_dummies(filtered_med_df_with_groups, columns=['MED_GROUP'], dtype=int)
med_df_dummies = med_df_dummies.groupby('case_no').agg('sum').reset_index()

In [249]:
adm_df_dummies = pd.get_dummies(filtered_adm_df_with_groups, columns=['MED_GROUP'], dtype=int)
adm_df_dummies = adm_df_dummies.groupby('case_no').agg('sum').reset_index()

In [250]:
merged_df = pd.merge(med_df_dummies, adm_df_dummies, on='case_no', how='right')

In [251]:
merged_df['MED_GROUP_Not specified'] = merged_df['MED_GROUP_Not specified_x'] + merged_df['MED_GROUP_Not specified_y']
merged_df['MED_GROUP_alimentary tract and metabolism'] = merged_df['MED_GROUP_alimentary tract and metabolism_x'] + merged_df['MED_GROUP_alimentary tract and metabolism_y']
merged_df['MED_GROUP_alimentary tract and metabolism ; blood and blood forming organs'] = merged_df['MED_GROUP_alimentary tract and metabolism ; blood and blood forming organs_x'] + merged_df['MED_GROUP_alimentary tract and metabolism ; blood and blood forming organs_y']
merged_df['MED_GROUP_antiinfectives for systemic use'] = merged_df['MED_GROUP_antiinfectives for systemic use_x'] + merged_df['MED_GROUP_antiinfectives for systemic use_y']
merged_df['MED_GROUP_antineoplastic and immunomodulating agents'] = merged_df['MED_GROUP_antineoplastic and immunomodulating agents_x'] + merged_df['MED_GROUP_antineoplastic and immunomodulating agents_y']
merged_df['MED_GROUP_blood and blood forming organs'] = merged_df['MED_GROUP_blood and blood forming organs_x'] + merged_df['MED_GROUP_blood and blood forming organs_y']
merged_df['MED_GROUP_cardiovascular system'] = merged_df['MED_GROUP_cardiovascular system_x'] + merged_df['MED_GROUP_cardiovascular system_y']
merged_df['MED_GROUP_genito-urinary system and sex hormones'] = merged_df['MED_GROUP_genito-urinary system and sex hormones_x'] + merged_df['MED_GROUP_genito-urinary system and sex hormones_y']
merged_df['MED_GROUP_nervous system'] = merged_df['MED_GROUP_nervous system_x'] + merged_df['MED_GROUP_nervous system_y']
merged_df['MED_GROUP_respiratory system'] = merged_df['MED_GROUP_respiratory system_x'] + merged_df['MED_GROUP_respiratory system_y']
merged_df['MED_GROUP_systemic hormonal preparations, excluding sex hormones and insulins'] = merged_df['MED_GROUP_systemic hormonal preparations, excluding sex hormones and insulins_x'] + merged_df['MED_GROUP_systemic hormonal preparations, excluding sex hormones and insulins_y']
merged_df['MED_GROUP_various'] = merged_df['MED_GROUP_various_x'] + merged_df['MED_GROUP_various_y']

In [254]:
cols_to_remove = ['MED_GROUP_Not specified_x', 'MED_GROUP_Not specified_y',
                  'MED_GROUP_alimentary tract and metabolism ; blood and blood forming organs_x',
                  'MED_GROUP_alimentary tract and metabolism ; blood and blood forming organs_y',
                  'MED_GROUP_antiinfectives for systemic use_x',
                  'MED_GROUP_antiinfectives for systemic use_y',
                  'MED_GROUP_antineoplastic and immunomodulating agents_x',
                  'MED_GROUP_antineoplastic and immunomodulating agents_y',
                  'MED_GROUP_blood and blood forming organs_x',
                  'MED_GROUP_blood and blood forming organs_y',
                  'MED_GROUP_cardiovascular system_x',
                  'MED_GROUP_cardiovascular system_y',
                  'MED_GROUP_genito-urinary system and sex hormones_x',
                  'MED_GROUP_genito-urinary system and sex hormones_y',
                  'MED_GROUP_nervous system_x', 'MED_GROUP_nervous system_y',
                  'MED_GROUP_respiratory system_x', 'MED_GROUP_respiratory system_y',
                  'MED_GROUP_systemic hormonal preparations, excluding sex hormones and insulins_x',
                  'MED_GROUP_systemic hormonal preparations, excluding sex hormones and insulins_y',
                 'MED_GROUP_various_x', 'MED_GROUP_various_y',
                  'MED_GROUP_alimentary tract and metabolism_x', 'MED_GROUP_alimentary tract and metabolism_y',
                  'MED_GROUP_Not specified', 'MED_GROUP_various',
                  'MED_GROUP_pending review']

In [255]:
merged_df.drop(columns=cols_to_remove, inplace=True)
merged_df.fillna(0, inplace=True)

In [256]:
percentage_of_zeros = ((merged_df == 0).mean() * 100).reset_index()
cols_to_remain = percentage_of_zeros.loc[percentage_of_zeros[0] < 99]['index'].tolist()
merged_df = merged_df[cols_to_remain]

In [136]:
merged_df.to_csv('data_for_model/med_df.csv')